## 1.背景

- **集体智慧**：为了创造新的想法，而将一群人的行为、偏好或思想组合在一起。
- **机器学习的局限**：机器学习算法受限于其在大量模式之上的归纳能力，且只能凭借已经见过的数据进行归纳。如果一个模式不同于算法先前所见的任何其他模式，那么它很有可能会被“误解”。
- **协作性过滤**：一个协作性过滤算法通常的做法是对一大群人进行搜索，并从中找出与我们品味相近的一群人。

## 2.搜集偏好

下面是一个涉及影评者及其对几部电影评分情况的字典：

In [1]:
from recommendations import critics
print(critics)
print("\nLisa Rose 对 Lady in the Water 的评分：",critics['Lisa Rose']['Lady in the Water'])
print("\nToby 评价过的电影及评分：", critics['Toby'])

{'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 'The Night Listener': 3.0}, 'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5}, 'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0, 'Superman Returns': 3.5, 'The Night Listener': 4.0}, 'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'The Night Listener': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 2.5}, 'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 2.0}, 'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5}, 'Toby': {'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman 

## 寻找相近的用户
搜集完人们的偏好数据之后，我们需要有一种方法来确定人们在品味方面的相似程度。为此，我们可以将每个人与所有其他人进行对比，并计算他们的**相似度评价值**。两套常用的计算相似度评价值的体系：**欧几里得距离**和**皮尔逊相关度**。

- **欧氏距离**：介绍略

In [2]:
# 通过欧氏距离计算两个人相似度评价的函数
from math import sqrt
def sim_distance(prefs, p1, p2):
    # 如果两人没有共同项，返回0
    sim = []
    for item in prefs[p1]:
        if item in prefs[p2]:
            sim.append(item)
            
    if len(sim) == 0:
        return 0
    
    # 计算欧氏距离
    distance = sqrt(sum([pow(prefs[p1][item]-prefs[p2][item], 2) for item in sim]))
    return 1/(1+distance) # 防止除数为0

In [3]:
sim_distance(critics, 'Lisa Rose', 'Gene Seymour')

0.29429805508554946

- **皮尔逊相关度**

该相关系数是判断两组数据与某一直线拟合程度的一种度量。它在数据不是很规范（normalized）的时候（比如，影评者对影片的评价总是相对于平均水平偏离很大时），会倾向于给出更好的结果。

其公式为：
$$P_{X,Y}=\cfrac{COV(X,Y)}{\sigma_x\sigma_y}
=\cfrac{E((X-\mu_X)(Y-\mu_Y))}{\sigma_x\sigma_y}
=\cfrac{E(XY)-E(X)E(Y)}{\sqrt{E(X^2)-E^2(X)}\sqrt{E(Y^2)-E^2(Y)}}$$

书中给出的公式为：
$$P_{X,Y}=\cfrac{\sum{XY}-\cfrac{\sum{X}\sum{Y}}{N}}{\sqrt{(\sum{X^2}-\cfrac{\sum^2{X}}{N})(\sum{Y^2}-\cfrac{\sum^2{Y}}{N})}}
=\cfrac{\cfrac{\sum{XY}-\cfrac{\sum{X}\sum{Y}}{N}}{N}}{\cfrac{\sqrt{(\sum{X^2}-\cfrac{\sum^2{X}}{N})(\sum{Y^2}-\cfrac{\sum^2{Y}}{N})}}{N}}
=\cfrac{\cfrac{\sum{XY}}{N}-\cfrac{\sum{X}}{N}\cfrac{\sum{Y}}{N}}{\sqrt{(\frac{\sum{X^2}}{N}-\frac{\sum^2{X}}{N^2})(\frac{\sum{Y^2}}{N}-\frac{\sum^2{Y}}{N^2})}}$$

两者是等价的，但书中给出的公式更便于计算

In [4]:
# 通过皮尔逊相关系数计算相似度
def sim_pearson(prefs, p1, p2):
    # 如果两人没有共同项，返回1
    sim = []
    for item in prefs[p1]:
        if item in prefs[p2]:
            sim.append(item)
    
    n = len(sim)
    if n == 0:
        return 1
    
    # 求和
    sum1 = sum([prefs[p1][item] for item in sim])
    sum2 = sum([prefs[p2][item] for item in sim])
    
    # 求平方和
    sum1_sq = sum([pow(prefs[p1][item], 2) for item in sim])
    sum2_sq = sum([pow(prefs[p2][item], 2) for item in sim])
    
    # 求乘积之和
    p_sum = sum([prefs[p1][item]*prefs[p2][item] for item in sim])
    
    # 计算相关系数
    num = p_sum - (sum1*sum2/n)
    den = sqrt((sum1_sq-pow(sum1,2)/n)*(sum2_sq-pow(sum2,2)/n))
    
    if den==0:
        return 0
    r = num/den
    return r

In [5]:
sim_pearson(critics, 'Lisa Rose', 'Gene Seymour')

0.39605901719066977

## 为评论者打分
既然已经有了对两个人进行比较的函数，下面可以编写函数，根据指定的人对每个人进行打分，并找出最接近的匹配结果。

In [6]:
def topMatches(prefs, person, n=5, sim_func=sim_pearson):
    '''
    perfs: 偏好字典
    n: 返回最接近的n个结果
    sim_func: 评价函数
    '''
    # 返回（相似度，名字）的元组
    scores = [(sim_func(prefs, person, other), other) for other in prefs if other != person] 
    scores.sort()
    scores.reverse()
    return scores[:n]

In [7]:
topMatches(critics, 'Toby', n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

## 推荐物品
我们大多数时候想要的不是找到一位趣味相投的影评人，而是一份影片的推荐。为此，我们需要通过一个经过加权的评价值为影片打分，影片最终的评分为每个影片人的评分在相似度上的加权平均。

In [8]:
def getRecommendations(prefs, person, sim_func=sim_pearson):
    res = {}
    sim_sum = {}
    for other in prefs:
        if other == person:
            continue
        sim = sim_func(prefs, person, other)
        
        if sim<=0:
            continue
        for item in prefs[other]:
            # 只对自己还未看过的影片评价
            if item not in prefs[person] or prefs[person][item] == 0:
                # 默认为0
                res.setdefault(item, 0)
                res[item] += sim*prefs[other][item]
                sim_sum.setdefault(item, 0)
                sim_sum[item] += sim
                
    rankings = [(v/sim_sum[k],k) for k,v in res.items()]
    
    rankings.sort()
    rankings.reverse()
    return rankings

In [9]:
getRecommendations(critics,'Toby')

[(3.3477895267131017, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.530980703765565, 'Just My Luck')]

In [10]:
getRecommendations(critics, 'Toby', sim_func=sim_distance) # 此处书上有误，distance没有取平方根

[(3.457128694491423, 'The Night Listener'),
 (2.778584003814924, 'Lady in the Water'),
 (2.422482042361917, 'Just My Luck')]

## 匹配商品
如果想了解哪些商品是彼此相近的，应该怎么做？我们可以通过查看那些人喜欢某一特定物品，以及这些人喜欢哪些其他物品来决定相似度。这和我们之前用来决定人和人之间相似度的方法是一样的，只需要将人和物品对换即可。

可能一下难以理解，下面简要说明：
- 相似度：根据**两人**对**不同影片**的评分计算 -> 根据**不同人**对**两部影片**的评分计算

- topMatch：找出最相似的人 -> 找出最相似的影片
- getRecommendations：根据**其他人与指定人相似度**的加权给指定人**没有看过的影片**打分 -> 根据**其他影片与指定影片相似度**的加权预测**没有看过这部影片的人**的评分

In [11]:
def transPrefs(prefs):
    res = {}
    for person in prefs:
        for item in prefs[person]:
            res.setdefault(item, {})
            res[item][person] = prefs[person][item]
    return res

In [12]:
print(transPrefs(critics))

{'Lady in the Water': {'Lisa Rose': 2.5, 'Gene Seymour': 3.0, 'Michael Phillips': 2.5, 'Mick LaSalle': 3.0, 'Jack Matthews': 3.0}, 'Snakes on a Plane': {'Lisa Rose': 3.5, 'Gene Seymour': 3.5, 'Michael Phillips': 3.0, 'Claudia Puig': 3.5, 'Mick LaSalle': 4.0, 'Jack Matthews': 4.0, 'Toby': 4.5}, 'Just My Luck': {'Lisa Rose': 3.0, 'Gene Seymour': 1.5, 'Claudia Puig': 3.0, 'Mick LaSalle': 2.0}, 'Superman Returns': {'Lisa Rose': 3.5, 'Gene Seymour': 5.0, 'Michael Phillips': 3.5, 'Claudia Puig': 4.0, 'Mick LaSalle': 3.0, 'Jack Matthews': 5.0, 'Toby': 4.0}, 'You, Me and Dupree': {'Lisa Rose': 2.5, 'Gene Seymour': 3.5, 'Claudia Puig': 2.5, 'Mick LaSalle': 2.0, 'Jack Matthews': 3.5, 'Toby': 1.0}, 'The Night Listener': {'Lisa Rose': 3.0, 'Gene Seymour': 3.0, 'Michael Phillips': 4.0, 'Claudia Puig': 4.5, 'Mick LaSalle': 3.0, 'Jack Matthews': 3.0}}


In [13]:
# 得到一组与《Superman Returns》最相近的影片
movies = transPrefs(critics)
topMatches(movies, 'Superman Returns')

[(0.6579516949597695, 'You, Me and Dupree'),
 (0.4879500364742689, 'Lady in the Water'),
 (0.11180339887498941, 'Snakes on a Plane'),
 (-0.1798471947990544, 'The Night Listener'),
 (-0.42289003161103106, 'Just My Luck')]

In [14]:
# 根据影片推荐人
getRecommendations(movies, 'Just My Luck')

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

In [15]:
# 上一个全是整数，比较有迷惑性
getRecommendations(movies, 'Lady in the Water')

[(3.610031066802182, 'Toby'), (3.4436241497684494, 'Claudia Puig')]

## 3.基于物品的过滤

现在我们要使用来自每一位用户的全部评分来构造数据集。这种方法对于数量以千计的用户或物品规模而言或许是没有问题的，但对于像Amazon这样有着**上百万客户和商品**的大型网站而言，将一个用户和**所有其他用户**进行比较，然后再对这位用户评过分的商品进行比较，其速度可能是无法忍受的。同样，一个商品销售量为数百万的网站，也许用户在偏好方面**彼此间很少会有重叠**，这可能会令用户的相似性判断变得十分困难。

目前为止我们所采用的技术被称为**基于用户的协作型过滤**（user-based collaborative filtering）。除此之外，还有一种可供选择的方法被称为**基于物品的协作型过滤**（item-based collaborative filtering）。在拥有大量数据集的情况下，基于物品的协作型过滤能够得出更好的结论，而且它**允许我们将大量计算任务预先执行**，从而使需要给予推荐的用户能够更快地得到他们所要的结果。

## 构造物品比较数据集
构造一个包含相近物品的完整数据集，构建完之后，我们就可以在需要的时候重复使用它。

In [16]:
def calculateSimilarItems(prefs, n=10):
    # 建立字典，以给出与这些物品最为相近的所有其他物品
    result={}
    
    # 以物品为中心对偏好字典实施倒置处理
    itemPrefs = transPrefs(prefs)
    c = 0
    for item in itemPrefs:
        c +=1
        if c%100 == 0: print("%d / %d" % (c, len(itemPrefs)))
        # 寻找最相近的物品
        scores = topMatches(itemPrefs, item, n=n, sim_func=sim_distance)
        result[item] = scores
    return result

In [17]:
itemsim = calculateSimilarItems(critics)
itemsim

{'Lady in the Water': [(0.4494897427831781, 'You, Me and Dupree'),
  (0.38742588672279304, 'The Night Listener'),
  (0.3483314773547883, 'Snakes on a Plane'),
  (0.3483314773547883, 'Just My Luck'),
  (0.2402530733520421, 'Superman Returns')],
 'Snakes on a Plane': [(0.3483314773547883, 'Lady in the Water'),
  (0.32037724101704074, 'The Night Listener'),
  (0.3090169943749474, 'Superman Returns'),
  (0.2553967929896867, 'Just My Luck'),
  (0.1886378647726465, 'You, Me and Dupree')],
 'Just My Luck': [(0.3483314773547883, 'Lady in the Water'),
  (0.32037724101704074, 'You, Me and Dupree'),
  (0.2989350844248255, 'The Night Listener'),
  (0.2553967929896867, 'Snakes on a Plane'),
  (0.20799159651347807, 'Superman Returns')],
 'Superman Returns': [(0.3090169943749474, 'Snakes on a Plane'),
  (0.252650308587072, 'The Night Listener'),
  (0.2402530733520421, 'Lady in the Water'),
  (0.20799159651347807, 'Just My Luck'),
  (0.1918253663634734, 'You, Me and Dupree')],
 'You, Me and Dupree': [

## 获得推荐

In [18]:
def getRecommendedItems(prefs, itemMatch, user):
    userRatings = prefs[user]
    scores={}
    totalSim = {}
    
    for item,rating in userRatings.items():
        for sim,item2 in itemMatch[item]:
            if item2 in userRatings: continue
            
            # 评分与相似度加权和
            scores.setdefault(item2,0)
            scores[item2]+=sim*rating
            
            # 相似度之和
            totalSim.setdefault(item2,0)
            totalSim[item2]+=sim
            
    rankings = [(score/totalSim[item], item) for item,score in scores.items()]
    
    rankings.sort()
    rankings.reverse()
    return rankings

In [19]:
getRecommendedItems(critics,itemsim,'Toby')

[(3.1667425234070894, 'The Night Listener'),
 (2.9366294028444346, 'Just My Luck'),
 (2.868767392626467, 'Lady in the Water')]

## 4.使用MovieLens数据集
MovieLens是一个涉及电影评价的真实数据集。我们只用关心数据集中的两个文件：u.item和u.data，前者包含了一组有关电影ID和片名的列表，后者则包含了以如下形式给出的实际评价情况：
```sh
196 242 3 881250949
186 302 3 891717742
22  377 1 878887116
...
```
此处的每一行数据都包含了一个用户ID、影片ID、用户对该片所给的评分，以及评价的时间。该数据集中包含了943名用户对1682部影片所作的评价，每位用户至少曾为20部影片做过评价。

In [20]:
import os

def loadMoiveLens(path='./data/movielens/'):
    # 获取影片标题
    movies = {}
    with open(os.path.join(path, 'u.item'), encoding='ISO-8859-1') as file:
        for line in file:
            id,title = line.split('|')[0:2]
            movies[id] = title
            
    # 加载数据
    prefs = {}
    with open(os.path.join(path, 'u.data'), encoding='ISO-8859-1') as file:
        for line in file:
            user,movieid,rating,ts = line.split('\t')
            prefs.setdefault(user,{})
            prefs[user][movies[movieid]]=float(rating)
    return prefs 

In [21]:
prefs = loadMoiveLens()
prefs['87']

{'Naked Gun 33 1/3: The Final Insult (1994)': 4.0,
 'Con Air (1997)': 4.0,
 'Sabrina (1995)': 4.0,
 'Waterworld (1995)': 4.0,
 'To Wong Foo, Thanks for Everything! Julie Newmar (1995)': 3.0,
 'Clueless (1995)': 4.0,
 'Jurassic Park (1993)': 5.0,
 'Brady Bunch Movie, The (1995)': 2.0,
 'Son in Law (1993)': 4.0,
 'Indiana Jones and the Last Crusade (1989)': 5.0,
 'Good, The Bad and The Ugly, The (1966)': 5.0,
 'Dead Poets Society (1989)': 5.0,
 'Dead Man Walking (1995)': 4.0,
 "Joe's Apartment (1996)": 2.0,
 'GoldenEye (1995)': 4.0,
 'M*A*S*H (1970)': 5.0,
 'Something to Talk About (1995)': 2.0,
 'Lightning Jack (1994)': 3.0,
 'Big Green, The (1995)': 3.0,
 'Cowboy Way, The (1994)': 3.0,
 "Ulee's Gold (1997)": 3.0,
 'Addams Family Values (1993)': 2.0,
 '2001: A Space Odyssey (1968)': 5.0,
 'Platoon (1986)': 3.0,
 'Return of the Pink Panther, The (1974)': 4.0,
 'Four Weddings and a Funeral (1994)': 5.0,
 'Under Siege (1992)': 4.0,
 'Ace Ventura: Pet Detective (1994)': 4.0,
 'Die Hard: Wit

In [22]:
# 基于用户评价过的影片推荐新影片
getRecommendations(prefs, '87')[0:30]

[(5.0, 'They Made Me a Criminal (1939)'),
 (5.0, 'Star Kid (1997)'),
 (5.0, 'Santa with Muscles (1996)'),
 (5.0, 'Saint of Fort Washington, The (1993)'),
 (5.0, 'Marlene Dietrich: Shadow and Light (1996) '),
 (5.0, 'Great Day in Harlem, A (1994)'),
 (5.0, 'Entertaining Angels: The Dorothy Day Story (1996)'),
 (5.0, 'Boys, Les (1997)'),
 (4.89884443128923, 'Legal Deceit (1997)'),
 (4.815019082242709, 'Letter From Death Row, A (1998)'),
 (4.800260666069042, 'Mrs. Dalloway (1997)'),
 (4.771240079753505, 'Leading Man, The (1996)'),
 (4.7321082983941425, 'Hearts and Minds (1996)'),
 (4.707354190896574, 'Dangerous Beauty (1998)'),
 (4.696244466490867, 'Pather Panchali (1955)'),
 (4.652397061026758, 'Lamerica (1994)'),
 (4.532337612572981, 'Innocents, The (1961)'),
 (4.527998574747076, 'Casablanca (1942)'),
 (4.512903125553784, 'Four Days in September (1997)'),
 (4.510270149719864, 'Everest (1998)'),
 (4.485151301801341, 'Wallace & Gromit: The Best of Aardman Animation (1996)'),
 (4.463287461

**改为基于物品的推荐**：

In [23]:
itemsim = calculateSimilarItems(prefs,n=50)

100 / 1664
200 / 1664
300 / 1664
400 / 1664
500 / 1664
600 / 1664
700 / 1664
800 / 1664
900 / 1664
1000 / 1664
1100 / 1664
1200 / 1664
1300 / 1664
1400 / 1664
1500 / 1664
1600 / 1664


In [24]:
getRecommendedItems(prefs,itemsim,'87')[0:30]

[(5.0, "What's Eating Gilbert Grape (1993)"),
 (5.0, 'Vertigo (1958)'),
 (5.0, 'Usual Suspects, The (1995)'),
 (5.0, 'Toy Story (1995)'),
 (5.0, 'Titanic (1997)'),
 (5.0, 'Sword in the Stone, The (1963)'),
 (5.0, 'Stand by Me (1986)'),
 (5.0, 'Sling Blade (1996)'),
 (5.0, 'Silence of the Lambs, The (1991)'),
 (5.0, 'Shining, The (1980)'),
 (5.0, 'Shine (1996)'),
 (5.0, 'Sense and Sensibility (1995)'),
 (5.0, 'Scream (1996)'),
 (5.0, 'Rumble in the Bronx (1995)'),
 (5.0, 'Rock, The (1996)'),
 (5.0, 'Robin Hood: Prince of Thieves (1991)'),
 (5.0, 'Reservoir Dogs (1992)'),
 (5.0, 'Police Story 4: Project S (Chao ji ji hua) (1993)'),
 (5.0, 'House of the Spirits, The (1993)'),
 (5.0, 'Fresh (1994)'),
 (5.0, 'Denise Calls Up (1995)'),
 (5.0, 'Day the Sun Turned Cold, The (Tianguo niezi) (1994)'),
 (5.0, 'Before the Rain (Pred dozhdot) (1994)'),
 (5.0, 'Assignment, The (1997)'),
 (5.0, '1-900 (1994)'),
 (4.875, "Ed's Next Move (1996)"),
 (4.833333333333333, 'Anna (1996)'),
 (4.8, 'Dark City 

**尽管构造物品的相似度字典花费了较长时间，但是推荐的过程几乎是在数据构造完毕后瞬间完成的。而且，获取推荐所花费的时间不会随着用户数量的增加而增加。**

**对于稀疏数据集，基于物品的过滤方法通常要优于基于用户的过滤方法，而对于密集数据集而言，两者的效果则几乎是一样的。基于用户的过滤方法更加易于实现，而且无需额外步骤，因此它通常更适用于规模较小的变化非常频繁的内存数据集。**